In [1]:
from utils import Score
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

In [2]:
model_ckpt = "/data/davide/models/t5/rankt5/flan-t5-xl-difference-total-30-ga-32/checkpoint-1/"
score_strategy = "difference"

get_score = getattr(Score, score_strategy)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [4]:
tokenizer.encode('true')

[1176, 1]

In [4]:
query = "What is the significance of the Turing Award in the field of computer science?"

documents = [
    "Title: Major Milestones in Computer Science. Content: Another significant recognition in the world of computing is the Turing Award. Named after Alan Turing, the father of theoretical computer science and artificial intelligence, the award is often dubbed the 'Nobel Prize of Computer Science.' Established in 1966, it is conferred annually by the Association for Computing Machinery (ACM) to an individual or a group who made major contributions to the field of computer science.",
    "Title: Alan Turing, a Brief Biography. Content: Despite his monumental contributions, Turing faced persecution in his lifetime due to his sexuality. However, his legacy lives on in various ways, one of which is the Turing Award. While this award bears his name and is given in the domain of computer science, its significance in terms of academic recognition remains underappreciated.",
    "Title: Notbale Awards in Various Disciplines. Content: Turning to the realm of computer science, one cannot forget the Turing Award. The Turing Award has been around for decades and is a prestigious accolade, honoring those who have pushed the boundaries of computer science research and innovation. This award is more than just a title; it carries with it a rich history and a commitment to recognizing excellence in computing.",
]

inputs = [
    f"Query: {query} Document: {document} Relevant: " 
    for document in documents
]

features = tokenizer(
    inputs, truncation = True, 
    return_tensors = "pt", max_length = 500, 
    padding = True,
)

input_ids = features.input_ids
attention_mask = features.attention_mask
decode_ids = torch.full((input_ids.size(0), 1),
                         model.config.decoder_start_token_id,
                         dtype=torch.long)

In [6]:
with torch.no_grad():
    output = model(
            input_ids = input_ids.to(device), 
            attention_mask = attention_mask.to(device), 
            decoder_input_ids = decode_ids.to(device),    
        ) 
    logits = output.logits[:,0,:]

scores = get_score(logits).cpu().tolist()

In [7]:
import pandas as pd

corpus = pd.read_csv("/data/davide/dragon/beir/nfcorpus/collection.tsv", sep="\t", nrows=10)
queries = pd.read_csv("/data/davide/dragon/beir/nfcorpus/queries.test.tsv", sep="\t", nrows=10)
qrels = pd.read_csv("/data/davide/dragon/beir/nfcorpus/qrels.test.tsv", sep=" ", nrows=10)

In [8]:
qrels

,PLAIN-2,0,MED-2427,2
0,PLAIN-2,0,MED-10,2
1,PLAIN-2,0,MED-2429,2
2,PLAIN-2,0,MED-2430,2
3,PLAIN-2,0,MED-2431,2
4,PLAIN-2,0,MED-14,2
5,PLAIN-2,0,MED-2432,2
6,PLAIN-2,0,MED-2428,1
7,PLAIN-2,0,MED-2440,1
8,PLAIN-2,0,MED-2434,1
9,PLAIN-2,0,MED-2435,1


In [4]:
queries

,PLAIN-3,Breast Cancer Cells Feed on Cholesterol
0,PLAIN-4,Using Diet to Treat Asthma and Eczema
1,PLAIN-5,Treating Asthma With Plants vs. Pills
2,PLAIN-6,How Fruits and Vegetables Can Treat Asthma
3,PLAIN-7,How Fruits and Vegetables Can Prevent Asthma
4,PLAIN-8,Our Tax Dollars Subsidize Unhealthy Foods
5,PLAIN-9,Reducing Arsenic in Chicken and Rice
6,PLAIN-10,How Contaminated Are Our Children?
7,PLAIN-14,"Blood Type Diet Perceived as ""Crass Fraud"""
8,PLAIN-15,Why Do Heart Doctors Favor Surgery and Drugs O...
9,PLAIN-16,Phytates in Beans: Anti-Nutrient or Anti-Cancer?
